<a href="https://colab.research.google.com/github/shauryay7/Youtube-Transcript/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **YouTube Transcript**

`language` use the language code to get the video.

`punctuation_model` values can be found at https://huggingface.co/oliverguhr/fullstop-punctuation-multilang-large#languages

## Runtime
Please go to `Runtime > Change runtime type > Select T4 GPU`
This will ensure best performance. Without a gpu, the punctuation will be very slow and can take minutes.

## Environment Variables

- `YOUTUBE_API_KEY`: Set up your Google API key for video information retrieval. You will need to create a Project in the google cloud for this and enable the YouTube v3 API.

In [1]:
!pip install youtube-transcript-api deepmultilingualpunctuation nltk google-api-python-client

In [2]:
url = 'https://youtu.be/Sx6dAx7dnXg?si=xBMYxgT8r0D2vQkT'  # Copy paste youtube link u want to transcript
language = 'en'
punctuated = True
output_dir = '.' # add /content/drive/MyDrive/ to save content in You Google Drive
filename = ""
punctuation_model = '' # get values from below link
verbose = True

`punctuation_model` values can be found at https://huggingface.co/oliverguhr/fullstop-punctuation-multilang-large#languages

In [3]:
from google.colab import drive # To store file in Drive

In [4]:
import os
import logging
import re
import math
import nltk
import youtube_transcript_api
from deepmultilingualpunctuation import PunctuationModel
import googleapiclient.discovery
import googleapiclient.errors

from google.colab import userdata
import warnings

In [5]:
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

logging.basicConfig(level=logging.INFO, force=True)
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
def clean_for_filename(title):
    cleaned_title = re.sub(r'[^\w\s\.\-\(\)\[\]]', '', title)
    return cleaned_title.strip()

def remove_music_tags(text):
    updated_text = re.sub(r'\[music\]', '', text, flags=re.IGNORECASE)
    return updated_text

def remove_period_after_hashes(text):
    return re.sub(r'(#\.|##\.)', lambda match: match.group(1)[:-1], text)

def remove_escape_sequences(text):
    # Some old videos contain escape sequences like \n.
    return re.sub(r'\\[nrtb]|\\r\n', '', text)

def remove_double_greater_than(text):
    cleaned_text = re.sub(r'>>', '', text)
    return cleaned_text

def add_punctuation(text, punctuation_model):
    if punctuation_model != "":
        model = PunctuationModel(model=punctuation_model)
    else:
        model = PunctuationModel()
        punctuated_text = model.restore_punctuation(text)
    return punctuated_text

def capitalize_sentences(sentences):
    capitalized_sentences = [sentence[0].upper() + sentence[1:] for sentence in sentences]
    return capitalized_sentences

def parse_youtube_url(url):
    video_id_match = re.search(r'(?:youtube\.com\/.*?[?&]v=|youtu\.be\/)([^"&?\/\s]{11})', url)
    if video_id_match:
        return video_id_match.group(1)
    else:
        raise ValueError('Invalid YouTube URL')

def parse_chapters(description):
    lines = description.split("\n")
    regex = re.compile(r"(\d{0,2}:?\d{1,2}:\d{2})")
    chapters = []

    for line in lines:
        matches = regex.findall(line)
        if matches:
            ts = matches[0]
            title = line.replace(ts, "").strip()
            title = re.sub(r'\d{0,2}:?\d{1,2}:\d{2}', '', title).strip().strip('-').strip().strip('-').strip()

            chapters.append({
                "timestamp": ts,
                "title": title,
            })

    return chapters

def get_transcript(video_id, language, video_info, verbose=True):
    transcript_list = youtube_transcript_api.YouTubeTranscriptApi.get_transcript(video_id, languages=[language])

    if video_info["title"] != "":
        transcript = f'# {video_info["title"]}\n\n'

    current_chapter_index = 0
    chapters = video_info["chapters"]
    logging.info(f"Transcript_List Length: {len(transcript_list)}, Chapter Length: {len(chapters)}")

    for i, line in enumerate(transcript_list):
        start_time = int(math.floor(line['start']))
        if 0 <= current_chapter_index < len(chapters):
            chapter_time = chapters[current_chapter_index]['timestamp']

            try:
                chapter_start = chapter_time.strip()
                chapter_start_seconds = sum(int(x) * 60 ** i for i, x in enumerate(reversed(chapter_start.split(':'))))
                chapters[current_chapter_index]["title"] = chapters[current_chapter_index]["title"].strip()
                buffer_time = 2

                if start_time >= chapter_start_seconds - buffer_time:
                    logging.info(f'\n\n## {chapters[current_chapter_index]["title"]}\n')
                    current_chapter_index += 1
            except Exception as e:
                logging.error(f"Error processing chapter timestamp: {chapter_time}")
                logging.error(f"Error details: {e}")

        line['text'] = remove_music_tags(line['text'])
        line['text'] = remove_escape_sequences(line['text'])
        line['text'] = remove_double_greater_than(line['text'])
        if line['text']:
          transcript += line['text'].strip() + ' '

        if verbose and i % 100 == 0:
            logging.info(f"Processed {i} lines out of {len(transcript_list)}")

    return transcript

def process_and_save_transcript(video_id, video_info, language, generate_punctuated, output_dir, filename, verbose, punctuation_model):
    try:
        raw_transcript = get_transcript(video_id, language, video_info, verbose)
        logging.info("Raw Transcript Length: %d", len(raw_transcript))

        if generate_punctuated:
            with_punctuation = add_punctuation(raw_transcript, punctuation_model)
            with_punctuation = remove_period_after_hashes(with_punctuation)
            logging.info("Punctuation Char Length: %d", len(with_punctuation))
            sentences = nltk.sent_tokenize(with_punctuation)
            logging.info("Sentences to process, (punctuated): %d", len(sentences))
        else:
            sentences = nltk.sent_tokenize(raw_transcript)
            logging.info("Sentences to process, (raw): %d", len(sentences))

        capitalized_sentences = capitalize_sentences(sentences)

        double_linesep = os.linesep + os.linesep
        capitalized_transcript = double_linesep.join(capitalized_sentences)
        output_path = os.path.join(output_dir, f'{filename}.md')

        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(capitalized_transcript)

        if generate_punctuated:
            logging.info(f'Punctuated transcript saved to {output_path}')
        else:
            logging.info(f'Raw transcript saved to {output_path}')

    except Exception as e:
        logging.error(f'Error: {e}')

def getVideoInfo (video_id):
  try:
    api_key =  userdata.get('YOUTUBE_API_KEY') # Replace with your actual API key
    youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)
    request = youtube.videos().list(part="id,snippet",
                                id = video_id
        )
    response = request.execute()
    title = response['items'][0]['snippet']['title']
    description = response['items'][0]['snippet']['description']
    data = {"title" : title, "chapters" : parse_chapters(description)}
    return data
  except Exception as e:
    logging.error(f'Error: {e}')
    return {"title": "", "chapters": []}

In [7]:
video_id = parse_youtube_url(url)
video_info = getVideoInfo(video_id)
filename = filename = filename or clean_for_filename(video_info["title"]) or clean_for_filename(video_id)

In [8]:
process_and_save_transcript(video_id, video_info, language, punctuated, output_dir, filename, verbose, punctuation_model)

INFO:root:Transcript_List Length: 380, Chapter Length: 0
INFO:root:Processed 0 lines out of 380
INFO:root:Processed 100 lines out of 380
INFO:root:Processed 200 lines out of 380
INFO:root:Processed 300 lines out of 380
INFO:root:Raw Transcript Length: 11980


config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/406 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
INFO:root:Punctuation Char Length: 12264
INFO:root:Sentences to process, (punctuated): 111
INFO:root:Punctuated transcript saved to ./iPhone 12 Unboxing Experience  MagSafe Demo.md


In [9]:
# Download the Generated File
from google.colab import files
files.download(os.path.join(output_dir, f'{filename}.md'))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>